In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#Load Datasets
customer_feedback = pd.read_csv("blinkit_customer_feedback.csv")
customers = pd.read_csv("blinkit_customers.csv")
delivery_performance = pd.read_csv("blinkit_delivery_performance.csv")
inventory = pd.read_csv("blinkit_inventory.csv")
inventory_new = pd.read_csv("blinkit_inventoryNew.csv")
marketing_performance = pd.read_csv("blinkit_marketing_performance.csv")
order_items = pd.read_csv("blinkit_order_items.csv")
orders = pd.read_csv("blinkit_orders.csv")
products = pd.read_csv("blinkit_products.csv")

# Check first few rows of a dataset
customer_feedback.head()

FileNotFoundError: [Errno 2] No such file or directory: 'blinkit_customer_feedback.csv'

In [9]:
customer_feedback.isnull().sum()

feedback_id          0
order_id             0
customer_id          0
rating               0
feedback_text        0
feedback_category    0
sentiment            0
feedback_date        0
dtype: int64

In [10]:
products.isnull().sum()

product_id           0
product_name         0
category             0
brand                0
price                0
mrp                  0
margin_percentage    0
shelf_life_days      0
min_stock_level      0
max_stock_level      0
dtype: int64

In [11]:
orders.isnull().sum()

order_id                  0
customer_id               0
order_date                0
promised_delivery_time    0
actual_delivery_time      0
delivery_status           0
order_total               0
payment_method            0
delivery_partner_id       0
store_id                  0
dtype: int64

In [12]:
order_items.isnull().sum()

order_id      0
product_id    0
quantity      0
unit_price    0
dtype: int64

In [13]:
marketing_performance.isnull().sum()

campaign_id          0
campaign_name        0
date                 0
target_audience      0
channel              0
impressions          0
clicks               0
conversions          0
spend                0
revenue_generated    0
roas                 0
dtype: int64

In [14]:
inventory_new.isnull().sum()

product_id        0
date              0
stock_received    0
damaged_stock     0
dtype: int64

In [15]:
inventory.isnull().sum()

product_id        0
date              0
stock_received    0
damaged_stock     0
dtype: int64

In [16]:
delivery_performance.isnull().sum()

order_id                    0
delivery_partner_id         0
promised_time               0
actual_time                 0
delivery_time_minutes       0
distance_km                 0
delivery_status             0
reasons_if_delayed       1902
dtype: int64

In [17]:
customers.isnull().sum()

customer_id          0
customer_name        0
email                0
phone                0
address              0
area                 0
pincode              0
registration_date    0
customer_segment     0
total_orders         0
avg_order_value      0
dtype: int64

In [20]:
#Fill missing delay reasons
delivery_performance.loc[:, "reasons_if_delayed"] = delivery_performance["reasons_if_delayed"].fillna("Unknown")

In [21]:
#Standardizing Date Format in inventory & inventory_new
inventory.loc[:, "date"] = pd.to_datetime(inventory["date"], format="%d-%m-%Y", errors="coerce")

In [24]:
#Removing Duplicates
inventory_new = inventory_new.drop_duplicates()

In [26]:
#Fix: Modify delivery_status Column Based on delivery_time_minutes
# Define threshold for "Strictly Delayed" (changeable as needed)
strict_delay_threshold = 10  # minutes

# Update the delivery_status column based on conditions
delivery_performance.loc[:, "delivery_status"] = delivery_performance["delivery_time_minutes"].apply(
    lambda x: "Early" if x < 0 else 
              "Strictly Delayed" if x > strict_delay_threshold else 
              "Late" if x > 0 else 
              "On Time"
)

In [27]:
#Ensuring Correct Pricing (price < mrp)
products.loc[products["price"] > products["mrp"], "price"] = products.loc[products["price"] > products["mrp"], "mrp"]

In [28]:
#Final Missing Values Check
datasets = {
    "customer_feedback": customer_feedback,
    "customers": customers,
    "delivery_performance": delivery_performance,
    "inventory": inventory,
    "inventory_new": inventory_new,
    "marketing_performance": marketing_performance,
    "order_items": order_items,
    "orders": orders,
    "products": products,
}

for name, df in datasets.items():
    print(f"Missing values in {name}:")
    print(df.isnull().sum(), "\n")

Missing values in customer_feedback:
feedback_id          0
order_id             0
customer_id          0
rating               0
feedback_text        0
feedback_category    0
sentiment            0
feedback_date        0
dtype: int64 

Missing values in customers:
customer_id          0
customer_name        0
email                0
phone                0
address              0
area                 0
pincode              0
registration_date    0
customer_segment     0
total_orders         0
avg_order_value      0
dtype: int64 

Missing values in delivery_performance:
order_id                 0
delivery_partner_id      0
promised_time            0
actual_time              0
delivery_time_minutes    0
distance_km              0
delivery_status          0
reasons_if_delayed       0
dtype: int64 

Missing values in inventory:
product_id        0
date              0
stock_received    0
damaged_stock     0
dtype: int64 

Missing values in inventory_new:
product_id        0
date              0

In [29]:
customer_feedback.to_csv("cleaned_customer_feedback.csv", index=False)

In [30]:
customers.to_csv("cleaned_customers.csv", index=False)

In [31]:
delivery_performance.to_csv("cleaned_delivery_performance.csv", index=False)

In [32]:
inventory.to_csv("cleaned_inventory.csv", index=False)

In [33]:
inventory_new.to_csv("cleaned_inventoryNew.csv", index=False)

In [34]:
marketing_performance.to_csv("cleaned_marketing_performance.csv", index=False)

In [35]:
order_items.to_csv("cleaned_order_items.csv", index=False)

In [36]:
orders.to_csv("cleaned_orders.csv", index=False)

In [37]:
products.to_csv("cleaned_products.csv", index=False)

In [38]:
orders["promised_delivery_time"] = pd.to_datetime(orders["promised_delivery_time"], errors="coerce")
orders["actual_delivery_time"] = pd.to_datetime(orders["actual_delivery_time"], errors="coerce")

In [39]:
# Calculate delivery time difference in minutes
orders["delivery_time_minutes"] = (
    (orders["actual_delivery_time"] - orders["promised_delivery_time"]).dt.total_seconds() / 60
)

In [40]:
# Define threshold for "Strictly Delayed"
strict_delay_threshold = 10  # minutes

# Apply classification logic
orders["delivery_status"] = orders["delivery_time_minutes"].apply(
    lambda x: "Early" if x < 0 else 
              "Strictly Delayed" if x > strict_delay_threshold else 
              "Late" if x > 0 else 
              "On Time"
)

In [41]:
orders[["promised_delivery_time", "actual_delivery_time", "delivery_time_minutes", "delivery_status"]].head(10)

,promised_delivery_time,actual_delivery_time,delivery_time_minutes,delivery_status
0,2024-07-17 08:52:01,2024-07-17 08:47:01,-5.0,Early
1,2024-05-28 13:25:29,2024-05-28 13:27:29,2.0,Late
2,2024-09-23 13:25:12,2024-09-23 13:29:12,4.0,Late
3,2023-11-24 16:34:56,2023-11-24 16:33:56,-1.0,Early
4,2023-11-20 05:17:39,2023-11-20 05:18:39,1.0,Late
5,2023-03-18 16:49:51,2023-03-18 16:48:51,-1.0,Early
6,2023-04-16 19:01:37,2023-04-16 19:02:37,1.0,Late
7,2024-03-31 06:37:48,2024-03-31 06:39:48,2.0,Late
8,2023-07-14 00:02:36,2023-07-14 00:05:36,3.0,Late
9,2023-08-09 01:37:30,2023-08-09 01:44:30,7.0,Late


In [42]:
orders.to_csv("cleaned_orders.csv", index=False)
print("✅ Cleaned orders dataset saved successfully.")

✅ Cleaned orders dataset saved successfully.
